В домашнем задании нужно решить задачу регрессии. В качестве датасета необходимо взять данные о недвижимости Калифорнии.
Целевая переменная – MedHouseVal. На полученных данных построить модель регрессии и дерево решений.

In [ ]:
from sklearn.datasets import fetch_california_housing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

1. Получите данные и загрузите их в рабочую среду. (Jupyter Notebook или другую).

In [ ]:
print(fetch_california_housing().DESCR)

In [ ]:
housing = fetch_california_housing()

In [ ]:
housing.data

In [ ]:
housing.target

In [ ]:
housing.feature_names

In [ ]:
# Преобразуем данные в таблицу

data = pd.DataFrame(data=housing.data, columns=housing.feature_names)
data['MedHouseVal'] = housing.target
data.head()

2. Проведите первичный анализ.
* Проверьте данные на пропуски. Удалите в случае обнаружения.
* Нормализуйте один из признаков.

In [ ]:
data.info()

In [ ]:
data['MedHouseVal'].value_counts().sort_values()

In [ ]:
data['MedHouseVal'].sort_values().unique()

In [ ]:
data['MedInc'].sort_values().unique()

In [ ]:
# Построим корреляционную матрицу

corr_matrix = data.corr()
corr_matrix

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(corr_matrix, annot=True, fmt='.2f')

plt.title('Корреляционная матрица признаков')

In [ ]:
# Рассмотрим распределение признаков на графике

data.hist(figsize=(15, 8), bins=30)
plt.tight_layout()
plt.show()  

3. Разделите выборку на обучающее и тестовое подмножества. 80% данных оставить на обучающее множество, 20% - на тестовое.

In [ ]:
X = data[['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']]
y = data['MedHouseVal']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

4. Обучите модель регрессии на обучающем множестве.

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

5. Для тестового множества предскажите целевую переменную и сравните с истинным значением, посчитав точность предсказания модели. Для этого используйте встроенную функцию score.

In [ ]:
y_test_pred = model.predict(X_test)
y_test_pred

In [ ]:
# Оценим R2 на тестовой выборке на исходных данных

r2 = r2_score(y_test, y_test_pred)
print(f'Тестовая выборка без преобразований: R2 = {r2:.3f}')

In [ ]:
# Построим график предсказаний целевой переменной MedHouseVal

plt.figure(figsize=(8,6))
plt.scatter(y_test, y_test_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r-')  # линия регрессии
plt.xlabel('Истинные значения MedHouseVal')
plt.ylabel('Предсказанные значения MedHouseVal')
plt.title('График предсказаний целевой переменной и истинных значений')
plt.show()

In [ ]:
# Преобразуем асимметричные распределения с помощью логарифмирования

features_scaler = ['MedInc', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup']

for feature in features_scaler:
    data[feature] = np.log1p(data[feature])

In [ ]:
X = data[['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']]
y = data['MedHouseVal']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Оценим распределение признаков на графике после логарифмирования

data.hist(figsize=(15, 8), bins=30)
plt.tight_layout()
plt.show()  

In [ ]:
# Обучим модель на преобразованных признаках

model = LinearRegression()
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)

In [ ]:
# Оценим R2 на тестовой выборке после преобразований log - метрика стала немного лучше

r2 = r2_score(y_test, y_test_pred)
print(f'Тестовая выборка логарифмирование: R2 = {r2:.3f}')

In [ ]:
# Оценим распределение MedHouseVal в зависимости от положения по Latitude, Longitude

plt.figure(figsize=(15, 8))

sns.scatterplot(
    x='Latitude', 
    y='Longitude', 
    data=data, 
    hue='MedHouseVal',  
    palette="coolwarm",       
    alpha=0.6       
)

plt.title('Распределение целевой переменной MedHouseVal')
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.legend(title='Median Value ($100,000)')
plt.show()

In [ ]:
# Подтвердим по геопозиции предположение относительно расположения у береговой линии

min_value = data['MedHouseVal'].min()

min_rows = data[data['MedHouseVal'] == min_value]
min_rows

In [ ]:
# Выделим новый столбец MedHouseVal_category с категориями стоимости домов

bins = pd.cut(data['MedHouseVal'], bins=5, labels=['Very_Low', 'Low', 'Average', 'High', 'Very_High'])
data['MedHouseVal_category'] = bins
data.head()

In [ ]:
data_bin = data.groupby('MedHouseVal_category')[['Latitude', 'Longitude']].mean().reset_index()
data_bin

In [ ]:
# Используем One-Hot Encoding для преобразования нового столбца

data_encoded = pd.get_dummies(data, columns=['MedHouseVal_category'])
data_encoded.head()

In [ ]:
data_encoded.columns

In [ ]:
# Обучим модель с учетом новых призаков по расположению у океана

X = data_encoded[['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'MedHouseVal_category_Very_Low',
       'MedHouseVal_category_Low', 'MedHouseVal_category_Average',
       'MedHouseVal_category_High', 'MedHouseVal_category_Very_High']]
y = data_encoded['MedHouseVal']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)

In [ ]:
# Значительно улучшили результат предсказаний модели

rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2 = r2_score(y_test, y_test_pred)
print(f'Тестовая выборка итог: RMSE = {rmse:.3f}, R2 = {r2:.3f}')

In [ ]:
# Построим график предсказаний целевой переменной MedHouseVal

plt.figure(figsize=(8,6))
plt.scatter(y_test, y_test_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r-')  # линия регрессии
plt.xlabel('Истинные значения MedHouseVal')
plt.ylabel('Предсказанные значения MedHouseVal')
plt.title('График предсказаний целевой переменной и истинных значений')
plt.show()

6. Обучите дерево решений на обучающем множестве.
* Повторите п. 5 для полученной модели.
* Визуализируйте часть дерева решений.

In [ ]:
X = housing.data
y = housing.target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Обучим дерево без ограничений

tree = DecisionTreeRegressor(random_state=42)
tree.fit(X_train, y_train)
y_train_pred = tree.predict(X_train)
y_test_pred = tree.predict(X_test)

In [ ]:
rmse_ = np.sqrt(mean_squared_error(y_train, y_train_pred))
r2_ = r2_score(y_train, y_train_pred)
print(f"Тренировочная выборка с ограничением глубины дерева: RMSE = {rmse_:.3f}, R2 = {r2_:.3f}")

In [ ]:
rmse_ = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_ = r2_score(y_test, y_test_pred)
print(f"Тестовая выборка с ограничением глубины дерева: RMSE = {rmse_:.3f}, R2 = {r2_:.3f}")

In [ ]:
# Без ограничений получаем переобучение, теперь попробуем ограничить глубину max_depth=3

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
tree = DecisionTreeRegressor(max_depth=3, random_state=42)
tree.fit(X_train, y_train)
y_test_pred = tree.predict(X_test)

In [ ]:
rmse_ = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_ = r2_score(y_test, y_test_pred)
print(f'Тестовая выборка с ограничением глубины дерева: RMSE = {rmse_:.3f}, R2 = {r2_:.3f}')

In [ ]:
# Визуализируем часть дерева решений 

plt.figure(figsize=(25, 10))
plot_tree(tree,
          feature_names=housing.feature_names,
          filled=True,
          rounded=True)
plt.show()

In [ ]:
# Построим график предсказаний целевой переменной MedHouseVal

plt.figure(figsize=(8,6))
plt.scatter(y_test, y_test_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r-')  # линия регрессии
plt.xlabel('Истинные значения MedHouseVal')
plt.ylabel('Предсказанные значения MedHouseVal')
plt.title('График предсказаний целевой переменной и истинных значений')
plt.show()

7. Оптимизируйте глубину дерева (max_depth). *Оптимизируйте ещё один параметр модели на выбор. Повторите п. 5 для полученной модели.

In [ ]:
# Увеличение глубины - увеличивает скор, оставим его без ограничений
# Ограничим число образцов в листовом узле

tree = DecisionTreeRegressor(min_samples_leaf=10, random_state=42)
tree.fit(X_train, y_train)
y_test_pred = tree.predict(X_test)

In [ ]:
rmse_ = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_ = r2_score(y_test, y_test_pred)
print(f'Тестовая выборка дерево с ограничением по листам: RMSE = {rmse_:.3f}, R2 = {r2_:.3f}')

In [ ]:
# Построим график предсказаний целевой переменной MedHouseVal

plt.figure(figsize=(8,6))
plt.scatter(y_test, y_test_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r-')  # линия регрессии
plt.xlabel('Истинные значения MedHouseVal')
plt.ylabel('Предсказанные значения MedHouseVal')
plt.title('График предсказаний целевой переменной и истинных значений')
plt.show()

In [ ]:
data_encoded.head()

In [ ]:
X = data_encoded[['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'MedHouseVal_category_Very_Low',
       'MedHouseVal_category_Low', 'MedHouseVal_category_Average',
       'MedHouseVal_category_High', 'MedHouseVal_category_Very_High']]
y = data_encoded['MedHouseVal']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
tree = DecisionTreeRegressor(min_samples_leaf=10, random_state=42)
tree.fit(X_train, y_train)
y_test_pred = tree.predict(X_test)

In [ ]:
rmse_ = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_ = r2_score(y_test, y_test_pred)
print(f'Тестовая выборка дерево на преобразованных признаках: RMSE = {rmse_:.3f}, R2 = {r2_:.3f}')

In [ ]:
# Построим график предсказаний целевой переменной MedHouseVal

plt.figure(figsize=(8,6))
plt.scatter(y_test, y_test_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r-')  # линия регрессии
plt.xlabel('Истинные значения MedHouseVal')
plt.ylabel('Предсказанные значения MedHouseVal')
plt.title('График предсказаний целевой переменной и истинных значений')
plt.show()

8. Сформулируйте выводы по проделанной работе.
* Сравните точность двух моделей.
* Напишите свое мнение, для каких задач предпочтительнее использовать обученные в работе модели? Какие у них есть плюсы и минусы?

### В данной задаче наибольшее влияние оказал фактор обработки входных параметров, а именно преобразование долготы и широты в один параметр. Результаты моделей при использовании преобразований близки.

In [ ]:
print(f'LinearRegression: RMSE:{rmse:.3f}, R2:{r2:.3f}; \nDecisionTreeRegressor: RMSE:{rmse_:.3f}, R2:{r2_:.3f}')
